# 第1节 – 聊天引导（Foundry Local）

此笔记本引导 Foundry Local，下载首选模型别名，并执行标准和流式聊天完成。


# 场景
本节介绍如何通过 Foundry Local 让本地小型语言模型进行响应的最低要求。您将完成以下任务：
- 安装 SDK / 客户端依赖项。
- 为选定的别名（默认：`phi-4-mini`）初始化 Foundry Local 管理器。
- 应用防御性猴子补丁以容忍模型元数据中的可选字段。
- 发送标准聊天完成请求。
- 按令牌逐步流式传输响应。

目标是验证您的本地运行时和网络路径，然后再转向 RAG、路由或代理。


### 说明：依赖安装
安装此最小聊天流程所需的 Python 包：
- `foundry-local-sdk`：管理本地模型和服务生命周期。
- `openai`：用于聊天完成的熟悉客户端抽象。
- `rich`：美观打印，便于更清晰的笔记本输出。

重新运行是安全的（幂等）。如果您的环境已经安装这些，可以跳过。


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 说明：核心导入
引入整个笔记本中使用的模块：
- `FoundryLocalManager` 用于与本地模型运行时交互。
- `OpenAI` 客户端，使我们可以重用熟悉的聊天完成 API 接口。
- `rich.print` 用于样式化输出。

这里不会发生网络调用——这只是准备命名空间。


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### 说明：管理器初始化与元数据修补
初始化选定别名的 `FoundryLocalManager`，并应用防御性猴子补丁，以优雅地处理 `promptTemplate` 可能为 `null` 的服务响应。

主要成果：
- 确认服务状态和端点。
- 列出缓存的模型（验证本地存储）。
- 解析别名的具体模型 ID（用于后续聊天调用）。

如果在原始服务元数据中遇到验证问题，此模式展示了如何在不分叉 SDK 的情况下进行清理。


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### 说明：基础聊天完成
创建一个指向本地 Foundry 端点的 `OpenAI` 兼容客户端，并执行一次非流式聊天完成。重点如下：
- 确保模型响应无错误。
- 验证延迟/输出格式。
- 保持 `max_tokens` 设置适中以节约资源。

如果失败，请重新检查 Foundry Local 服务是否正在运行以及别名是否正确解析。


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### 说明：流式聊天完成
展示了令牌流式传输以改善感知延迟和交互式用户体验。循环会在增量变化到达时打印出来：
- 对于聊天用户界面来说，早期的部分输出非常重要。
- 允许您测量令牌吞吐量与完整完成延迟之间的关系。

您可以调整此模式以累积令牌、更新进度小部件或在生成过程中中止。


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于重要信息，建议使用专业人工翻译。我们对因使用此翻译而产生的任何误解或误读不承担责任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
